In [1]:
import os
import pandas as pd
from bs4 import BeautifulSoup
import requests as r
from pprint import pprint

In [2]:
url = "https://losangeles.craigslist.org/d/for-sale/search/sss?query=snowboard&sort=rel"
response = r.get(url)
response

<Response [200]>

In [3]:
html_soup = BeautifulSoup(response.text, 'html.parser')

#get the macro-container for the housing posts
posts = html_soup.find_all('li', class_= 'result-row')
print(type(posts)) #to double check that I got a ResultSet
print(len(posts)) #to double check I got 120 (elements/page)
posts[0]

<class 'bs4.element.ResultSet'>
120


<li class="result-row" data-pid="7404893646" data-repost-of="6367300070">
<a class="result-image gallery" data-ids="3:00W0W_E1rogklv7qz_0lG0t2,3:00v0v_ftIvBZMl4fWz_0lG0t2,3:01313_5m76Wowiuybz_0lG0t2" href="https://losangeles.craigslist.org/sfv/spo/d/mission-hills-dakine-snowboard-140-with/7404893646.html">
<span class="result-price">$100</span>
</a>
<div class="result-info">
<span class="icon icon-star" role="button">
<span class="screen-reader-text">favorite this post</span>
</span>
<time class="result-date" datetime="2021-11-28 12:35" title="Sun 28 Nov 12:35:00 PM">Nov 28</time>
<h3 class="result-heading">
<a class="result-title hdrlnk" data-id="7404893646" href="https://losangeles.craigslist.org/sfv/spo/d/mission-hills-dakine-snowboard-140-with/7404893646.html" id="postid_7404893646">Dakine Snowboard 140 with  bindings</a>
</h3>
<span class="result-meta">
<span class="result-price">$100</span>
<span class="result-hood"> (Granada hills san fernando valley )</span>
<span class="result

In [4]:
next_page = r.get(f"https://losangeles.craigslist.org{html_soup.find('a',class_='button next')['href']}")

In [5]:
test_soup = BeautifulSoup(next_page.text, 'html.parser')
test_soup
posts = test_soup.find_all('li', class_= 'result-row')
# posts

* Step 1
    * Get the first page
* Step 2
    * get the data
* Step 3 
    * save the data in a dictionary
* Step 4
    * save the dictionary to a mongodb database
* Step 5
    * go to the next page
* Step 6
    * repeat previous steps until the last page is reached

In [6]:
url = "https://losangeles.craigslist.org/d/for-sale/search/sss?query=snowboard&sort=rel"
list_of_data = []
response = r.get(url)
html_soup = BeautifulSoup(response.text, 'html.parser')
for i in range(20):
    posts = html_soup.find_all('li', class_= 'result-row')
    # posts_prime = posts[0]
    # pprint(posts_prime)
    # print(posts_prime.find('a', class_= "result-title hdrlnk").title) # title
    # print(posts_prime.find('span', class_= "result-price").text) # price
    # print(posts_prime.find('span', class_= "result-hood").text) # location
    # print(posts_prime.find('a', class_= "result-title hdrlnk",href=True)['href']) # title
    # print("+"*100)
    for p in posts:
#         print(p.find('a', class_= "result-title hdrlnk").text) # title
#         print(p.find('span', class_= "result-price").text) # price
#         print(p.find('span', class_= "result-hood").text) # location
#         print(p.find('a', class_= "result-title hdrlnk",href=True)['href']) # Link
        
        
        try:
            temp_dict = {
                "post_id": p.find('a', class_="result-title hdrlnk")['id'],
                "title": p.find('a', class_= "result-title hdrlnk").text,
                "price": float(str(p.find('span', class_= "result-price").text).replace('$','').replace(',','')),
                "location": str(p.find('span', class_= "result-hood").text).replace(' (','').replace(' )',''),
                "link": p.find('a', class_= "result-title hdrlnk",href=True)['href']
            }
            list_of_data.append(temp_dict)
            # print(temp_dict)
        except:
            pass
        # print("="*100)
    try:
        response = r.get(f"https://losangeles.craigslist.org{html_soup.find('a',class_='button next')['href']}")
        html_soup = BeautifulSoup(response.text, 'html.parser')
    except:
        print("Finished scraping")
        break
print('+=+='*100)
print(len(list_of_data))

Finished scraping
+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=
474


In [8]:
df = pd.DataFrame(list_of_data)
df

,post_id,title,price,location,link
0,postid_7404893646,Dakine Snowboard 140 with bindings,100.0,Granada hills san fernando valley,https://losangeles.craigslist.org/sfv/spo/d/mi...
1,postid_7402218394,Morrow Snowboard 148 cm with morrow bindings,125.0,Granada hills san fernando valley,https://losangeles.craigslist.org/sfv/spo/d/mi...
2,postid_7410755840,Ride Snowboard 115 Cm and small Solomon bindings,125.0,Granada hills san fernando valley,https://losangeles.craigslist.org/sfv/spo/d/mi...
3,postid_7407043907,WOMENS SNOWBOARD BOOTS,160.0,Arcadia san gabriel valley,https://losangeles.craigslist.org/sgv/spo/d/ar...
4,postid_7413745357,"Women’s Snowboard, Bindings, and Boots",200.0,central LA 213/323,https://losangeles.craigslist.org/lac/spo/d/lo...
...,...,...,...,...,...
469,postid_7406161068,Snowboards with bindings,120.0,Encino san fernando valley,https://losangeles.craigslist.org/sfv/spo/d/en...
470,postid_7411626803,Snowboards,300.0,Frog Town central LA 213/323,https://losangeles.craigslist.org/lac/spo/d/lo...
471,postid_7408311060,Bird Scooter ES-1 Segway ES-1 Electric Scooter...,399.0,LA County westside-southbay-310,https://losangeles.craigslist.org/wst/sgd/d/lo...
472,postid_7405832903,Toyota Tacoma Roof Rack,199.0,Los Angeles westside-southbay-310,https://losangeles.craigslist.org/wst/ptd/d/re...
